In [15]:
from graphdatascience import GraphDataScience

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight') 

# neo4j desktop (5.3.0, m1)

In [16]:
host = "bolt://localhost:7687"
user = "neo4j"
password= "j4oenj4oen"

gds = GraphDataScience(host, auth=(user, password))
print(gds.version())

2.5.1


# 1. Import Data into Neo4j

In [17]:
query = """
LOAD CSV WITH HEADERS FROM 'https://github.com/neo4j-examples/nlp-knowledge-graph/raw/master/import/articles.csv' AS row
RETURN row
LIMIT 10;
"""
gds.run_cypher(query)

,row
0,{'uri': 'https://dev.to/lirantal/securing-a-no...
1,{'uri': 'https://dev.to/setevoy/neo4j-running-...
2,{'uri': 'https://dev.to/divyanshutomar/introdu...
3,{'uri': 'https://dev.to/zaiste/15-git-commands...
4,{'uri': 'https://dev.to/alexjitbit/removing-fi...
5,{'uri': 'https://dev.to/michelemauro/atlassian...
6,{'uri': 'https://dev.to/shirou/back-up-prometh...
7,{'uri': 'https://dev.to/ionic/farewell-phonega...
8,{'uri': 'https://dev.to/rootsami/rancher-kuber...
9,{'uri': 'https://dev.to/jignesh_simform/compar...


# 2. Load HTML

In [18]:
query = """
MERGE (a:Article {uri: "https://dev.to/setevoy/neo4j-running-in-kubernetes-e4p"})
WITH a
CALL apoc.load.html(a.uri, {body: 'body div.spec__body p', title: 'h1', time: 'time'}) 
YIELD value
UNWIND value.body AS item
WITH a,
     apoc.text.join(collect(item.text), '') AS body,
     value.title[0].text AS title,
     value.time[0].attributes.datetime AS date
SET a.body = body, a.title = title, a.datetime = datetime(date)
RETURN a;
"""
gds.run_cypher(query)

,a
0,"(datetime, body, title, uri)"


In [19]:
query = """
CALL apoc.periodic.iterate(
  "LOAD CSV WITH HEADERS FROM 'https://github.com/neo4j-examples/nlp-knowledge-graph/raw/master/import/articles.csv' AS row
   RETURN row",
  "MERGE (a:Article {uri: row.uri})
   WITH a
   CALL apoc.load.html(a.uri, {
     body: 'body div.spec__body p',
     title: 'h1',
     time: 'time'
   })
   YIELD value
   UNWIND value.body AS item
   WITH a,
        apoc.text.join(collect(item.text), '') AS body,
        value.title[0].text AS title,
        value.time[0].attributes.datetime AS date
   SET a.body = body , a.title = title, a.datetime = datetime(date)",
  {batchSize: 5, parallel: true}
)
YIELD batches, total, timeTaken, committedOperations
RETURN batches, total, timeTaken, committedOperations;
"""
gds.run_cypher(query)

,batches,total,timeTaken,committedOperations
0,7,32,11,32


# (postprocessing) free up memory

In [20]:
query = """
MATCH (n) DETACH DELETE n
"""
gds.run_cypher(query)
gds.close()